In [1]:
%run input/Format.ipynb
!rm output/dynamic_delta_fits/*
import ROOT as root
from array import array
import math as math
root.gErrorIgnoreLevel = root.kFatal
import multiprocessing as mp
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 0
is_sigmalised = 0
sim = 0
first_run = 1
Min_Entries = 4000
N_files = 1
Do_save = 1

In [3]:
isigma = 2
N_layers = 8
layer_bins =[0,1,4,5,8,9,12,13]
N_hists = 4
layers = ["1-2 layer east","1-2 layer west","2-3 layer east","2-3 layer west",\
          "2-4 layer east","2-4 layer west","3-4 layer east","3-4 layer west",
          "3 layer east pos","3 layer west pos","3 layer east neg","3 layer west neg",\
          "4 layer east pos","4 layer west pos","4 layer east neg","4 layer west neg",\
          "20-40%","40-60%","60-80%","80-93%"]
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_path="input/"
file_names="my-m_ee_Run14AuAu_93rd_new_19904_544runs" 
#m_ee_Run14AuAu_92nd_new_19900_1065runs m_ee_Run14AuAu_93rd_new_19904_100runs
#m_ee_Run14AuAu_84th_new_19850_778runs m_ee_Run14AuAu_83rd_new_19842_1065runs m_ee_Run14AuAu_85th_new_19858_1065runs m_ee_Run14AuAu_87th_new_19865_518runs
hist_select_3D_names = ["dphi_hist_el_dynamic","dthe_hist_el_dynamic","sdphi_hist_el_dynamic","sdthe_hist_el_dynamic"]
iOptames=["phi","theta"]

In [4]:
if sim :
    file_path="../sim/output/Newembed/dca/" 
    file_names="piminus_50M_new_v8"#piminus_50M_v7 piminus_50M_v8 piminus_50M_v9

In [5]:
hists_read = []

infile = root.TFile.Open(file_path+file_names+f".root", "read")
for ilayer in layer_bins:
    hist_select_3D0 = []
    for ihist in range(len(hist_select_3D_names)):
        hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{ilayer}"))
        hist_select_3D0[-1].SetDirectory(root.nullptr)
        hist_select_3D0[-1].Add(infile.Get(hist_select_3D_names[ihist]+f"_{ilayer+2}"))
    hists_read.append(hist_select_3D0)

infile.Close()

In [6]:
delta_z = 3
z_bins = [i for i in range(8,30,delta_z)]
z_means = array('d', [i-20+delta_z/2 for i in z_bins])
z_error = array('d', [0.5*delta_z for i in z_bins])
z_bin_edges = [i for i in range(8,33,delta_z)]
print(z_bins,z_means, z_error,z_bin_edges)
pad_limits = [[1.8,4.4],[-1.2,1.3]]
N_phi = 6
phi_fit_range = [[[1.9,2.2],[2.2,2.6],[2.6,2.9],[2.9,3.3],[3.3,3.7],[3.7,4.3]],\
                 [[-1.2,-0.7],[-0.7,-0.2],[-0.2,0.0],[0.0,0.3],[0.3,0.7],[0.7,1.2]]]#26 33 37
keff=[0.001,0.001,0.0005,0.0005,0.0005,0.0005,0.00025,0.00025]
keff = [(9-6*is_sigmalised)*keff[i] for i in range(len(keff))]
if sim: keff = [keff[i]/100 for i in range(len(keff))]
R_layers = [2.64,5.22,5.22,11.7,16.6]
phi_fit_range = [[[phi_fit_range[0][i][0], phi_fit_range[0][i][1]+0.1] for i in range(len(phi_fit_range[0]))],\
                 [[phi_fit_range[1][i][0], phi_fit_range[1][i][1]+0.1] for i in range(len(phi_fit_range[1]))]]
print(phi_fit_range)

[8, 11, 14, 17, 20, 23, 26, 29] array('d', [-10.5, -7.5, -4.5, -1.5, 1.5, 4.5, 7.5, 10.5]) array('d', [1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5]) [8, 11, 14, 17, 20, 23, 26, 29, 32]
[[[1.9, 2.3000000000000003], [2.2, 2.7], [2.6, 3.0], [2.9, 3.4], [3.3, 3.8000000000000003], [3.7, 4.3999999999999995]], [[-1.2, -0.6], [-0.7, -0.1], [-0.2, 0.1], [0.0, 0.4], [0.3, 0.7999999999999999], [0.7, 1.3]]]


In [7]:
def sdphi_fitter(ihits, sigma=isigma):

    iHist = hists_read[ihits][iOption0+is_sigmalised*2]

    Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, phi_local, phi_local_err, int_fg = [], [], [], [], [], [], []

    for iz in range(len(z_bin_edges)-1):

        phi_z =  array( 'd' )
        phi_z_err =  array( 'd' )
        Mean_z =  array( 'd' )
        Sigma_z =  array( 'd' )
        Mean_z_err =  array( 'd' )
        Sigma_z_err =  array( 'd' )
        int_fg_local =  array( 'd' )

        N_ybins_loc = iHist.GetNbinsY()
        for iy in range(N_ybins_loc):
            proj_local = iHist.ProjectionX(iHist.GetName()+f"_{iz}_{iy}", iy+1, iy+1, z_bin_edges[iz], z_bin_edges[iz+1]-1)

            phi = iHist.GetYaxis().GetBinCenter(iy+1)
            phi_z.append(phi)
            phi_z_err.append(iHist.GetYaxis().GetBinWidth(iy+1)/2.)

            c1 = root.TCanvas("c1"+proj_local.GetName(),"c1"+proj_local.GetName(),720,360)
            gaus0 = root.TF1("gaus0"+proj_local.GetName(), "gaus", -0.025, 0.025)
            proj_local.Draw()
            proj_local.Fit(gaus0, "Q", "", -0.025, 0.025)
            gaus1 = root.TF1("gaus1"+proj_local.GetName(), "gaus", gaus0.GetParameter(1)-0.002,gaus0.GetParameter(1)+0.002)
            proj_local.Fit(gaus1, "Q", "", gaus0.GetParameter(1)-0.006,gaus0.GetParameter(1)+0.006)            
            gaus = root.TF1("gaus"+proj_local.GetName(), "gaus", gaus1.GetParameter(1)-gaus1.GetParameter(2)*2,gaus1.GetParameter(1)+gaus1.GetParameter(2)*2)
            proj_local.Fit(gaus, "Q", "", gaus1.GetParameter(1)-gaus1.GetParameter(2)*1.99,gaus1.GetParameter(1)+gaus1.GetParameter(2)*1.99)



            if proj_local.GetEntries() < Min_Entries or abs(gaus.GetParameter(1))>0.006:
                Mean_z.append(0)
                Sigma_z.append(0.002)
                Mean_z_err.append(0.003)
                Sigma_z_err.append(0.003)
                int_fg_local.append(1)
                continue
            
            c1.Update()
            if Do_save: c1.SaveAs("output/dynamic_delta_fits/" + proj_local.GetName() + ".png")
            Mean_z.append(gaus.GetParameter(1))
            Sigma_z.append(gaus.GetParameter(2))
            #Mean_z_err.append(min(gaus.GetParError(1)*1000, gaus.GetParameter(2)/3))
            #Sigma_z_err.append(min(gaus.GetParError(2), 0.05*gaus.GetParameter(2)))


            int_fg_local.append(gaus.Integral(gaus.GetParameter(1)-0.002,gaus.GetParameter(1)+0.002)+0.0001)
            #int_bg_local.append(2*gaus.Integral(gaus.GetParameter(1)+sigma*gaus.GetParameter(2),0.0025))
            Mean_z_err.append(keff[ihist]/abs(int_fg_local[-1])**0.5)
            Sigma_z_err.append(keff[ihist]/abs(int_fg_local[-1])**0.5)

            #m2_pos.Draw("same") m2_neg.Draw("same")
            #c1.SaveAs( "output/dynamic_delta_fits/" + m2_name + ".png" )
        Mean_local.append(Mean_z)
        Mean_local_err.append(Mean_z_err)
        Sigma_local.append(Sigma_z)
        Sigma_local_err.append(Sigma_z_err)
        phi_local.append(phi_z)
        phi_local_err.append(phi_z_err)
        int_fg.append(int_fg_local)

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, phi_local, phi_local_err, int_fg]


In [8]:
Ntr = N_layers
pool = mp.Pool(Ntr)
with pool:
    output_array = pool.map(sdphi_fitter, [i for i in range(Ntr)])
pool.close()

In [9]:
print(len(output_array[0][0]))

8


In [10]:
Means, Means_err, Sigmas, Sigmas_err, phis, phis_err, int_fg = [], [], [], [], [], [], []
for i in range(N_layers):
    Means.append(output_array[i][0])
    Means_err.append(output_array[i][1])
    Sigmas.append(output_array[i][2])
    Sigmas_err.append(output_array[i][3])
    phis.append(output_array[i][4])
    phis_err.append(output_array[i][5])
    int_fg.append(output_array[i][6])
    
#print("Means", len(Means), len(Means[0]), len(Means[0][1]))

In [11]:
g_sigma = []
h2=[]
c3 = root.TCanvas(f"c3",f"c3",1400,125*N_layers)
c3.Divide(2,int(N_layers/2),0,0)
Min, Max = 0., 0.003
names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
legends =[]
for ilayer in range(N_layers):
    c3.cd(ilayer+1)
    g_sigma_layer = []
    h2.append(Format_Pad_old(pad_limits[ilayer%2][0], pad_limits[ilayer%2][1], Min, Max, "#phi [rad]", "#sigma#"+iOptames[iOption0] + " [mrad]", 1.1, 1.1, 0.06, 0.06, "",0.15,0.15,0.,0.1))
    root.gPad.SetTopMargin(0.0)
    root.gPad.SetRightMargin(0.0)
    h2[-1].Draw()
    legends.append(root.TLegend(0.55,0.5,0.95,0.98,layers[ilayer]))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    for ized in range(len(Sigmas[ilayer])):
        n_phi = len(Sigmas[ilayer][ized])
        if n_phi<1: continue
        g_sigma_layer.append(root.TGraphErrors(n_phi, phis[ilayer][ized], Sigmas[ilayer][ized],phis_err[ilayer][ized], Sigmas_err[ilayer][ized]))
        Format_Graph(g_sigma_layer[-1],20,1, ized, 3, ized,1,1)
        g_sigma_layer[-1].Draw("same p")
    g_sigma.append(g_sigma_layer)
    legends[-1].Draw()
        
    
    #g_sigma_f.append(g_sigma_layer)
#g_sigma.append(g_sigma_f)
c3.Draw()
#c3.SaveAs(f"output/note/sigma_{ilayer}{iOption0}{is_sigmalised}.pdf")

In [12]:
fit_mean_functions = []
for ilayer in range(N_layers):
    fit_functions = []
    for ized in range(len(z_bins)):
        fit_zed = []
        for iphi in range(N_phi):
            if not first_run:
                fit_zed.append(root.TF1(f"fit_functions_mean_iter_{ized}_"+layers[ilayer]+f"_{iphi}","[0]/10000*sin(x)+[1]/10000*cos(x)+[2]/10000",-1.5,4.5))
                fit_zed[-1].SetParameters(0.0, 0.0, 0.0)
            else:
                fit_zed.append(root.TF1(f"fit_functions_mean_iter_{ized}_"+layers[ilayer]+f"_{iphi}","[0]/10000*sin(x + 0.3/[3]*sin(x) + 0.04/[3]*cos(x))+[1]/10000*cos(x + 0.3/[3]*sin(x) + 0.04/[3]*cos(x))+[2]/10000",phi_fit_range[iOption0][iphi][0],phi_fit_range[iOption0][iphi][1]))
                fit_zed[-1].SetParameters(0.0, 0.0, 0.0, 0.0)
                fit_zed[-1].FixParameter(3,R_layers[int(ilayer/2)])

            fit_zed[-1].FixParameter(2,0.00)
            fit_zed[-1].SetParLimits(0, -75, 75)
            fit_zed[-1].SetParLimits(1, -75, 75)
            #fit_functions[-1].FixParameter(1,0.00)
            fit_zed[-1].SetLineColor(ized+1)
        fit_functions.append(fit_zed)
    fit_mean_functions.append(fit_functions)

In [13]:
g_mean = []
h2=[]
c3 = root.TCanvas(f"c3",f"c3",1400,125*N_layers)
c3.Divide(2,int(N_layers/2),0,0)
Min, Max = -0.005, 0.005
names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
legends =[]
for ilayer in range(N_layers):
    arm = ilayer%2
    c3.cd(ilayer+1)
    g_mean_layer = []
    h2.append(Format_Pad_old( pad_limits[ilayer%2][0], pad_limits[ilayer%2][1], Min, Max, "#phi [rad]", "#LT#Delta#"+iOptames[iOption0]+"#GT [mrad]", 1.1, 1.1, 0.06, 0.06, "",0.15,0.15,0.,0.1))
    root.gPad.SetTopMargin(0.0)
    root.gPad.SetRightMargin(0.0)
    h2[-1].Draw()
    legends.append(root.TLegend(0.2,0.8,0.45,0.98,layers[ilayer]))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    for ized in range(len(Means[ilayer])):
        n_phi = len(Means[ilayer][ized])
        if n_phi<1: continue
        g_sigma_layer.append(root.TGraphErrors(n_phi, phis[ilayer][ized], Means[ilayer][ized],phis_err[ilayer][ized], Means_err[ilayer][ized]))
        Format_Graph(g_sigma_layer[-1],20,1, ized+1, 3, ized+1,1,1)
        g_sigma_layer[-1].Draw("same p")
        for iphi in range(N_phi):
            g_sigma_layer[-1].Fit(fit_mean_functions[ilayer][ized][iphi],"QR+", "", phi_fit_range[arm][iphi][0],phi_fit_range[arm][iphi][1])
    g_sigma.append(g_sigma_layer)
    legends[-1].Draw()
        
    
    #g_sigma_f.append(g_sigma_layer)
#g_sigma.append(g_sigma_f)
c3.Draw()
#c3.SaveAs(f"output/real_mean.png")

In [14]:
parameters = []
par_errors = []
for ilayer in range(N_layers):
    parameters_layer, par_errors_layer = [], []
    for ized in range(len(z_bins)):
        parameters_zed, par_errors_zed = [], []
        for iphi in range(N_phi):
            parameters_phi, par_errors_phi = array('d'), array('d')
            for ipar in range(3):
                if ipar==0:
                    parameters_phi.append(fit_mean_functions[ilayer][ized][iphi].GetParameter(ipar))
                    par_errors_phi.append(fit_mean_functions[ilayer][ized][iphi].GetParError(ipar))
                if ipar==1:
                    parameters_phi.append(fit_mean_functions[ilayer][ized][iphi].GetParameter(ipar))
                    par_errors_phi.append(fit_mean_functions[ilayer][ized][iphi].GetParError(ipar))
                if ipar==2:
                    parameters_phi.append(fit_mean_functions[ilayer][ized][iphi].GetParameter(ipar))
                    par_errors_phi.append(fit_mean_functions[ilayer][ized][iphi].GetParError(ipar))
            parameters_zed.append(parameters_phi)
            par_errors_zed.append(par_errors_phi)
        parameters_layer.append(parameters_zed)
        par_errors_layer.append(par_errors_zed)
    parameters.append(parameters_layer)
    par_errors.append(par_errors_layer)

In [15]:
c2 = root.TCanvas(f"c2",f"c2",1400,125*N_layers)
c2.Divide(2,int(N_layers/2),0,0)
graphs_par = []
h2 = []
legends = []
for ilayer in range(N_layers):
    c2.cd(ilayer+1)
    h2.append(Format_Pad_old(-11,11, -50, 50, "z [cm]", "Parameter value [mrad]", 1.1, 1.1, 0.06, 0.06, "",0.15,0.15,0.,0.1))
    root.gPad.SetTopMargin(0.0)
    root.gPad.SetRightMargin(0.0)
    h2[-1].Draw()
    n_zed = len(z_bins)
    legends.append(root.TLegend(0.55,0.5,0.95,0.98,layers[ilayer]))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0) 
    for ipar in range(0,2):
        # Collect parameter values and errors for this layer and parameter
        for iphi in range(N_phi):
            par_vals = [parameters[ilayer][ized][iphi][ipar] for ized in range(n_zed)]
            par_errs = [par_errors[ilayer][ized][iphi][ipar] for ized in range(n_zed)]
            graphs_par.append(root.TGraphErrors(n_zed, z_means, array('d', par_vals), z_error, array('d', par_errs)))
            graphs_par[-1].SetMarkerStyle(20 + ipar%3)
            graphs_par[-1].SetMarkerColor(1 + iphi)
            graphs_par[-1].SetLineColor(1 + iphi)
            if iphi == 0:
                graphs_par[-1].SetTitle(f"{layers[ilayer]}: Parameter {ipar} vs z")
                graphs_par[-1].Draw("same p")
            else:
                graphs_par[-1].Draw("P SAME")
    legends[ilayer].AddEntry(graphs_par[-3], "Parameter 0", "p")
    legends[ilayer].AddEntry(graphs_par[-2], "Parameter 1", "p")
    legends[ilayer].AddEntry(graphs_par[-1], "Parameter 2", "p")    
    legends[ilayer].Draw()
c2.Draw()

In [16]:
# Print parameters as a C++ array initialization
if True:
    print(f"const float VTX_seg_alignment[{len(parameters)-2}][{n_zed}][{N_phi}][3] = {{")
    for ilayer in range(len(parameters)):
        if ilayer ==4 or ilayer == 5:  # Skip layers 4 and 5
            continue
        print("  {")
        for ized in range(n_zed):
            print("    {")
            for iphi in range(N_phi):
                vals = parameters[ilayer][ized][iphi]
                print(f"      {{{vals[0]:.4g}, {vals[1]:.4g}, {vals[2]:.4g}}}", end="")
                if iphi != N_phi - 1:
                    print(",", end="")
                print()
            print("    }" + ("," if ized != n_zed - 1 else ""))
        print("  }" + ("," if ilayer != len(parameters) - 1 else ""))
    print("};")

const float VTX_seg_alignment[6][8][6][3] = {
  {
    {
      {-16.09, -11.07, 0},
      {18.51, 12.25, 0},
      {-3.516, 0.2409, 0},
      {11.04, 3.015, 0},
      {4.196, 3.628, 0},
      {3.634, 3.814, 0}
    },
    {
      {9.29, 26.87, 0},
      {29.46, 19.06, 0},
      {-19, -4.772, 0},
      {5.244, -0.02131, 0},
      {6.846, -2.196, 0},
      {1.182, 1.7, 0}
    },
    {
      {12.61, 24.37, 0},
      {24.18, 13.02, 0},
      {-7.51, -2.943, 0},
      {7.124, -0.678, 0},
      {6.679, -3.535, 0},
      {0.1828, 1.708, 0}
    },
    {
      {13.78, 25.86, 0},
      {19.67, 10.95, 0},
      {-20.57, -6.491, 0},
      {13.66, -1.993, 0},
      {13.77, -8.582, 0},
      {-0.7683, -0.7868, 0}
    },
    {
      {0.1803, 5.254, 0},
      {7.895, 3.253, 0},
      {-26.78, -7.173, 0},
      {9.843, -1.746, 0},
      {15.25, -8.828, 0},
      {-0.294, -0.8222, 0}
    },
    {
      {0.8646, 15.28, 0},
      {4.774, 2.803, 0},
      {-60.87, -15.95, 0},
      {12.16, -2.443, 0},
      

In [17]:
# Example for plotting means in 3D for ilayer=0
ilayer = 0
g3d = root.TGraph2D()
point = 0
print(len(z_bins), len(Means[ilayer]), len(Means[ilayer][0]))
for ized, z in enumerate(z_bins):
    n_phi = len(Means[ilayer][ized])
    for iphi in range(n_phi):
        phi = phis[ilayer][ized][iphi]
        mean = Means[ilayer][ized][iphi]
        g3d.SetPoint(point, phi, z_means[ized], mean)
        point += 1
# Define the fit function: ([0]+[1]*zed)*sin(phi)+([2]+[3]*zed)*cos(phi)+[4]
fit_func = root.TF2("fit_func", "([0]+[1]*y)/10000*sin(x)+([2]+[3]*y)/10000*cos(x)+[4]/1000", -2, 4, min(z_means), max(z_means))
fit_func.SetParameters(0, 0, 0, 0, 0)  # You can tune these

c3d = root.TCanvas("c3d", "3D Mean", 800, 600)


g3d.SetTitle("Mean vs #phi and z;#phi;z;Mean")
g3d.SetMarkerStyle(20)
g3d.SetMarkerColor(root.kBlue)
# Fit the 3D graph
g3d.Fit(fit_func, "Q")  # "Q" for quiet

# Draw the points and the fit surface
g3d.Draw("P0 surf1")
fit_func.Draw("same surf1")

#c3d.Draw()

8 8 60


In [18]:
# Print the fit parameters and their errors for the 3D fit function
for i in range(5):
    par = fit_func.GetParameter(i)
    err = fit_func.GetParError(i)
    print(f"Parameter [{i}]: {par:.2g} ± {err:.2g}")

Parameter [0]: -0.64 ± 0.18
Parameter [1]: -0.1 ± 0.026
Parameter [2]: -0.14 ± 0.17
Parameter [3]: -0.051 ± 0.025
Parameter [4]: -0.0038 ± 0.012


In [19]:
# Fill a TH3D with (phi, z, mean) and fit 2D slices with the given function
layers3D = [0]
ilayer = layers3D[0]  # Example for the first layer in 3D
# Define binning
n_bins_phi = 20
n_bins_z = len(z_means)
phi_min, phi_max = pad_limits[ilayer%2][0], pad_limits[ilayer%2][1]
z_min, z_max = min(z_means), max(z_means)
mean_min, mean_max = -0.002, 0.002

th3 = root.TH3D("th3", "Mean vs #phi and z;#phi [rad];z [cm];#LT#Delta#phi#GT [mrad]", n_bins_phi, pad_limits[ilayer%2][0], pad_limits[ilayer%2][1], int(n_bins_z), z_min, 0, 100, mean_min, mean_max)
# Fill TH3D
for ilayer in layers3D:
    for ized, z in enumerate(z_bins):
        n_phi = len(Means[ilayer][ized])
        for iphi in range(n_phi):
            phi = phis[ilayer][ized][iphi]
            mean = Means[ilayer][ized][iphi]
            mean_err = int_fg[ilayer][ized][iphi]**0.5
            th3.Fill(phi, z_means[ized], mean, int_fg[ilayer][ized][iphi])
            #print(th3.FindBin(phi, z_means[ized], mean))
            ibin = th3.FindBin(phi, z_means[ized], mean)
            th3.SetBinError(ibin, th3.GetBinContent(ibin)**0.5)  # Set the error as the square root of the content

c1 = root.TCanvas("c1", "3D Histogram", 800, 600)
c1.SetLeftMargin(0.)
th3.Draw("LEGO2")  # Draw the 3D histogram
if False:#for first stupid run
    fit_func = root.TF2("fit_func", "([0]+[1]*y)/10000*sin(x + 0.3/[5]*sin(x) + 0.04/[5]*cos(x))+([2]+[3]*y)/10000*cos(x + 0.3/[5]*sin(x) + 0.04/[5]*cos(x))+[4]/1000", phi_min, phi_max, min(z_means), max(z_means))
    fit_func.FixParameter(5, R_layers[int(ilayer/2)])  # Fix the constant term to zero for the fit
else:
    fit_func = root.TF2("fit_func", "([0]+[1]*y)/10000*sin(x)+([2]+[3]*y)/10000*cos(x)+[4]/1000", phi_min, phi_max, min(z_means), max(z_means))
    fit_func.SetParameters(0, 0, 0, 0, 0)  # You can tune these
fit_func.FixParameter(4, 0)  # Fix the constant term to zero for the fit
th3.Fit(fit_func, "Q")  # Fit the 3D histogram
fit_func.Draw("same surf1")
c1.Draw()

# fit_results now contains the fit parameters for each z slice

In [20]:
print(f"{-fit_func.GetParameter(0):.3g} , {-fit_func.GetParameter(1):.3g}")
print(f"{-fit_func.GetParameter(2):.3g} , {-fit_func.GetParameter(3):.3g}")

0.948 , 0.155
3.64 , 0.572


In [21]:
#print(f"chi2/ndf = {fit_func.GetChisquare():.2f} / {fit_func.GetNDF()} = {fit_func.GetChisquare()/fit_func.GetNDF():.2f}")
# Print the fit parameters and their errors for the 3D fit function
for i in range(5):
    par = fit_func.GetParameter(i)
    err = fit_func.GetParError(i)
    print(f"Parameter [{i}]: {par:.2g} ± {err:.2g}")

Parameter [0]: -0.95 ± 1
Parameter [1]: -0.15 ± 0.24
Parameter [2]: -3.6 ± 0.5
Parameter [3]: -0.57 ± 0.11
Parameter [4]: 0 ± 0


In [22]:
#print(fit_func.Eval(3.14, 0))  # Example evaluation at phi=3.0, z=20
# Print the equation for dphi(z, phi) with obtained fit parameters
print(
    f"dphi = ({fit_func.GetParameter(0):.6g} + {fit_func.GetParameter(1):.6g}*z)/10000 * sin(phi) + "
    f"({fit_func.GetParameter(2):.6g} + {fit_func.GetParameter(3):.6g}*z)/10000 * cos(phi) + "
    f"{fit_func.GetParameter(4)/1000:.6g}"
)
print(f"x + ({fit_func.GetParameter(0):.3g} + {fit_func.GetParameter(1):.3g}*z)/10000")
print(f"y + ({fit_func.GetParameter(2):.3g} + {fit_func.GetParameter(3):.3g}*z)/10000")

dphi = (-0.948344 + -0.154798*z)/10000 * sin(phi) + (-3.63847 + -0.572414*z)/10000 * cos(phi) + 0
x + (-0.948 + -0.155*z)/10000
y + (-3.64 + -0.572*z)/10000


In [23]:
print(f"x + ({fit_func.GetParameter(0):.3g} + {fit_func.GetParameter(1):.3g}*z)/10000 + r * sin( {fit_func.GetParameter(4):.3g}/1000)*sin(phi)")
print(f"y + ({fit_func.GetParameter(2):.3g} + {fit_func.GetParameter(3):.3g}*z)/10000 + r * sin( {fit_func.GetParameter(4):.3g}/1000)*cos(phi)")

x + (-0.948 + -0.155*z)/10000 + r * sin( 0/1000)*sin(phi)
y + (-3.64 + -0.572*z)/10000 + r * sin( 0/1000)*cos(phi)


## Formula for Alignment Fit

The function used for fitting the mean shift as a function of azimuthal angle (φ) and longitudinal position (z) is:

<div align="center">

Δφ(z, φ) = ([0] + [1]·z)/10000 · sin(φ) + ([2] + [3]·z)/10000 · cos(φ) + [4]/1000

</div>

**where:**
- [0], [1], [2], [3], [4] are fit parameters extracted from the data,
- z is the longitudinal coordinate (e.g., along the beam axis),
- φ is the azimuthal angle.

---

### Why use this function for a 3D histogram fit?

- **Physical Motivation:**  
  Misalignments of detector layers often manifest as sinusoidal modulations in the measured hit positions as a function of φ, with possible linear dependence on z due to tilts or twists.

- **Parameter Extraction:**  
  Fitting this function to the 3D histogram of mean residuals (mean shift vs. φ and z) allows extraction of alignment parameters:
    - The coefficients of sin(φ) and cos(φ) (and their z-dependence) correspond to translations and tilts of the detector layer in the transverse plane.
    - The constant term can represent a global offset.

- **Usage:**  
  After fitting, the obtained parameters can be used to correct (align) the detector geometry, improving tracking and vertexing performance.